<a href="https://colab.research.google.com/github/fedraameneiro/DataScientistFinalProject/blob/main/Proyecto_final_Ciencia_de_Datos_FedraAmeneiro_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto diabetes



In [ ]:
#librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import plotly as pl
import plotly.express as px


In [ ]:
#obtener el archivo csv de drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/file/d/18ymyIaVZ8FK7WhyGltLrJbfx2nyvzjtK/view?usp=drive_link'

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('diabetes.csv')

In [ ]:
df=pd.read_csv('diabetes.csv', sep=';')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80,0,1,never,"25,19","6,6",140,0
1,Female,54,0,0,No Info,"27,32","6,6",80,0
2,Male,28,0,0,never,"27,32","5,7",158,0
3,Female,36,0,0,current,"23,45",5,155,0
4,Male,76,1,1,current,"20,14","4,8",155,0


Tenemos 4 variables categoricas y la variable objetivo. Y 3 variables numericas.

Antes de nada voy a chequear aquellas  variables numericas que aparecen como objetos y los voy a modificar. bmi y HbA1c_level deberían ser enteros. Les voy a cambiar la coma por un punto.

In [ ]:
df.bmi=df.bmi.str.replace(',','.')
df.HbA1c_level=df.HbA1c_level.str.replace(',','.')
df.bmi=df["bmi"].astype(float)
df.HbA1c_level=df["HbA1c_level"].astype(float)
df.dtypes

gender                  object
age                      int64
hypertension             int64
heart_disease            int64
smoking_history         object
bmi                    float64
HbA1c_level            float64
blood_glucose_level      int64
diabetes                 int64
dtype: object

In [ ]:
df_diabetes=df[df.diabetes==1]
df_nodiabetes=df[df.diabetes==0]
df_diabetes.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
6,Female,44,0,0,never,19.31,6.5,200,1
26,Male,67,0,1,not current,27.32,6.5,200,1
38,Male,50,1,0,current,27.32,5.7,260,1
40,Male,73,0,0,former,25.91,9.0,160,1
53,Female,53,0,0,former,27.32,7.0,159,1


Eliminamos aquellas muestras con genero other.

In [ ]:
df['gender'].unique()

array(['Female', 'Male', 'Other'], dtype=object)

In [ ]:
df=df[df.gender!='Other']

Vemos la correlación entre variables numéricas y confirmamos que las mejores variables que se relacionan con la diabetes son el nivel HbA1c_level y el nivel de glucosa en sangre.

In [ ]:
# Importa las bibliotecas necesarias
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X=df[['age', 'bmi','HbA1c_level','blood_glucose_level']]
y=df['diabetes']

# Normaliza las características (es importante para las redes neuronales)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir el conjunto de datos en conjuntos de entrenamiento y testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Construir el modelo de red neuronal
modelo = Sequential()
modelo.add(Dense(32, input_dim=4, activation='relu'))
modelo.add(Dense(16, activation='relu'))
modelo.add(Dense(1, activation='sigmoid'))



In [ ]:
# Compila el modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Entrena el modelo con los datos de entrenamiento. epoch=1000
modelo.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evalúa el modelo con los datos de prueba
precisionRedesNeuronales = modelo.evaluate(X_test, y_test) [1]
print(f'Precisión del modelo: {precisionRedesNeuronales}')


Epoch 1/100
2000/2000 [==============================] - 12s 5ms/step - loss: 0.1334 - accuracy: 0.9558 - val_loss: 0.1091 - val_accuracy: 0.9636
Epoch 2/100
2000/2000 [==============================] - 10s 5ms/step - loss: 0.1032 - accuracy: 0.9644 - val_loss: 0.0985 - val_accuracy: 0.9666
Epoch 3/100
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0945 - accuracy: 0.9677 - val_loss: 0.0919 - val_accuracy: 0.9687
Epoch 4/100
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0901 - accuracy: 0.9696 - val_loss: 0.0895 - val_accuracy: 0.9703
Epoch 5/100
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0876 - accuracy: 0.9709 - val_loss: 0.0874 - val_accuracy: 0.9719
Epoch 6/100
2000/2000 [==============================] - 5s 3ms/step - loss: 0.0861 - accuracy: 0.9713 - val_loss: 0.0871 - val_accuracy: 0.9718
Epoch 7/100
2000/2000 [==============================] - 4s 2ms/step - loss: 0.0855 - accuracy: 0.9718 - val_loss: 0.0856 - val

Voy a empezar metiendo datos que ya tenemos y revisamos si efectivamente predice de la manera correcta:

In [ ]:
#valor_glucosa=140
#valor_bmi=26.1
#valor_edad=21
#valorHbA1c=5.8
#diabetes=0

def es_diabetico(valor_glucosa, valor_bmi, valor_edad, valorHbA1c)

# Puedes usar el modelo entrenado para hacer predicciones en nuevos datos
paciente= np.array([[valor_edad, valor_bmi, valorHbA1c, valor_glucosa]])
nuevos_datos_scaled = scaler.transform(paciente)

prediccion_diabetes_paciente = modelo.predict(nuevos_datos_scaled)

# Convierte las probabilidades en predicciones binarias usando un umbral (por ejemplo, 0.5)
prediccion_binaria =  (prediccion_diabetes_paciente >= 0.5).astype(int)
# Mapea las predicciones binarias a etiquetas más descriptivas
resultado = "Diabetes" if prediccion_binaria[0] == 1 else "No Diabetes"
print(f'Predicción para paciente : {resultado}')
prediccion_diabetes_paciente
return resultado

1/1 [==============================] - 0s 85ms/step
Predicción para paciente : No Diabetes


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[0.01117145]], dtype=float32)